In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [28]:
!pip install mlflow

In [29]:
# import mlflow
# from mlflow.tracking import MlflowClient

# client = MlflowClient()

# experiment = client.get_experiment_by_name("House_Price_Regression_Correlation")
# if not experiment:
#     raise Exception(f"Experiment 'House_Price_Regression_Correlation' not found")

# experiment_id = experiment.experiment_id

# runs = client.search_runs(
#     experiment_ids=[experiment_id],
#     order_by=["attributes.start_time DESC"],
#     max_results=1
# )

# if not runs:
#     raise Exception(f"No runs found in experiment 'Y'")

# latest_run = runs[0]
# run_id = latest_run.info.run_id

# artifact_name = "my_list.json"
# local_path = f"/kaggle/working/{artifact_name}"

# downloaded_path = mlflow.artifacts.download_artifacts(
#     run_id=run_id,
#     artifact_path=artifact_name,
#     dst_path=local_path
# )

# print(f"Latest run ID: {run_id}")
# print(f"Downloaded artifact '{artifact_name}' to: {downloaded_path}")

In [30]:
import numpy as np
import pandas as pd
import mlflow
import os
import warnings
warnings.filterwarnings('ignore')

mlflow.set_tracking_uri("https://dagshub.com/dshan21/ML_ASS_1.mlflow")

test_file_path = "../input/house-prices-advanced-regression-techniques/test.csv"
test_df = pd.read_csv(test_file_path)
for_res = test_df.copy()
print(f"Test dataset shape: {test_df.shape}")

Test dataset shape: (1459, 80)


In [31]:
def engineer_features(df):
    import mlflow
    from mlflow.tracking import MlflowClient
    import os
    import json
    import numpy as np
    import requests
    import shutil
    
    try:
        df = df.copy()
        
        quality_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, np.nan: 0}
        df['ExterQualNum'] = df['ExterQual'].map(quality_map)
        df['ExterCondNum'] = df['ExterCond'].map(quality_map)
        df['BsmtQualNum'] = df['BsmtQual'].map(quality_map)
        df['BsmtCondNum'] = df['BsmtCond'].map(quality_map)
        df['GarageQualNum'] = df['GarageQual'].map(quality_map)
        df['GarageCondNum'] = df['GarageCond'].map(quality_map)
        
        df['OverallScore'] = df['OverallQual'] * df['OverallCond']
        df['ExterScore'] = df['ExterQualNum'] + df['ExterCondNum']
        df['GarageScore'] = df['GarageQualNum'] + df['GarageCondNum']
        df['BsmtScore'] = df['BsmtQualNum'] + df['BsmtCondNum']

        df.fillna(df.mode().iloc[0], inplace=True)
        df = df.reset_index(drop=True)
        my_list = {}
        highest_value_mapping = {}
        
        high_value_path = ""
        list_path = ""

        client = MlflowClient()
        
        experiment = client.get_experiment_by_name("House_Price_Regression_Object_OHE")
        if not experiment:
            raise Exception(f"Experiment 'House_Price_Regression_Correlation' not found")
        
        experiment_id = experiment.experiment_id
        
        runs = client.search_runs(
            experiment_ids=[experiment_id],
            order_by=["attributes.start_time DESC"],
            max_results=1
        )
        
        if not runs:
            raise Exception(f"No runs found in experiment 'Y'")
        
        latest_run = runs[0]
        run_id = latest_run.info.run_id
        
        artifact_name = "highest_value_mapping.json"
        local_path = f"/kaggle/working/{artifact_name}"
        
        high_value_path = mlflow.artifacts.download_artifacts(
            run_id=run_id,
            artifact_path=artifact_name,
            dst_path=local_path
        )
        

        experiment = client.get_experiment_by_name("House_Price_Regression_Correlation")
        if not experiment:
            raise Exception(f"Experiment 'House_Price_Regression_Correlation' not found")
        
        experiment_id = experiment.experiment_id
        
        runs = client.search_runs(
            experiment_ids=[experiment_id],
            order_by=["attributes.start_time DESC"],
            max_results=1
        )
        
        if not runs:
            raise Exception(f"No runs found in experiment 'Y'")
        
        latest_run = runs[0]
        run_id = latest_run.info.run_id
        
        artifact_name = "my_list.json"
        local_path = f"/kaggle/working/{artifact_name}"
        
        list_path = mlflow.artifacts.download_artifacts(
            run_id=run_id,
            artifact_path=artifact_name,
            dst_path=local_path
        )


        with open(high_value_path, "r") as f:
            highest_value_mapping = json.load(f)
            print("Successfully loaded highest_value_mapping")
            
        with open(list_path, "r") as f:
            my_list = json.load(f)
            print("Successfully loaded HIghest_corr")

        
        for colName, highest_value in highest_value_mapping.items():
            if colName in df.columns:
                df[colName + '_num'] = np.where(df[colName] == highest_value, 1, 0)
                df.drop(columns=[colName], inplace=True)
            else:
                print(f"Warning: Column {colName} not found in the DataFrame, skipping...")

        df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']
        df['TotalBath'] = df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath'])

        df['AgeAtSale'] = df['YrSold'] - df['YearBuilt']
        df['YearsSinceRemodel'] = df['YrSold'] - df['YearRemodAdd']
        df['GarageAge'] = df['YrSold'] - df['GarageYrBlt']
        df['GarageAge'] = df['GarageAge'].fillna(-1)
        
        
        
        df['TotalPorchSF'] = (
            df['OpenPorchSF'] +
            df['EnclosedPorch'] +
            df['3SsnPorch'] +
            df['ScreenPorch']
        )
        
        df['HasPorch'] = (df['TotalPorchSF'] > 0).astype(int)
        df['HasDeck'] = (df['WoodDeckSF'] > 0).astype(int)

        df['HasBasement'] = (df['TotalBsmtSF'] > 0).astype(int)
        df['TotalFinishedBsmtSF'] = df['BsmtFinSF1'] + df['BsmtFinSF2']
        df['TotalUsableBsmtSF'] = df['TotalFinishedBsmtSF'] + df['BsmtUnfSF']

        df['HasGarage'] = (df['GarageArea'] > 0).astype(int)
        df['GarageCapacityValue'] = df['GarageCars'] * df['GarageArea']
        df['GarageCapacityValue'] = df['GarageCapacityValue'].fillna(0)

        
        # print("LEEST", my_list)
        # print("DICT", highest_value_mapping)
        
        columns_to_keep = list(my_list.keys())
        if "SalePrice" in columns_to_keep:
            columns_to_keep.remove("SalePrice")

        if "Id" in df.columns:
            columns_to_keep.append("Id")
        
        # print("LEEST", columns_to_keep)
        
        existing_columns = [col for col in columns_to_keep if col in df.columns]
        df = df[existing_columns]
        
    except Exception as e:
        print("ragac gatrola")

    return df

In [32]:
import pandas as pd
import numpy as np
import mlflow.pyfunc
from IPython.display import FileLink

test_engineered = engineer_features(test_cleaned)

print(test_engineered.info())

test_features = test_engineered.drop(columns=['Id'])


print(test_engineered.head())

print("Loading model from MLflow Model Registry...")
model_name = "HousePriceRegressionModel"
model_version = 19

try:
    model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
    print(f"Successfully loaded {model_name} version {model_version}")
    print(type(model))
    print("Training columns:", list(test_engineered.columns))
    print("Test columns:    ", list(test_features.columns))


    
    predictions = model.predict(test_features)

    print("Raw prediction range:", predictions.min(), "-", predictions.max())
    print("Final price range:", predictions.min(), "-", predictions.max())
    
    # print("AAAAAAAA")
    submission = pd.DataFrame({
        'Id': for_res['Id'],
        'SalePrice': predictions
    })
    # print("BBBBBBB")

    
    submission.to_csv('/kaggle/working/submission.csv', index=False)
    FileLink('/kaggle/working/submission.csv')
    
    print("Submission file created successfully!")
    
except Exception as e:
    print(f"Error during model loading or prediction: {e}")




Successfully loaded highest_value_mapping
Successfully loaded HIghest_corr
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OverallQual          1459 non-null   int64  
 1   TotalSF              1459 non-null   float64
 2   GrLivArea            1459 non-null   int64  
 3   GarageCapacityValue  1459 non-null   float64
 4   ExterQualNum         1459 non-null   int64  
 5   GarageCars           1459 non-null   float64
 6   GarageArea           1459 non-null   float64
 7   TotalBath            1459 non-null   float64
 8   TotalUsableBsmtSF    1459 non-null   float64
 9   TotalBsmtSF          1459 non-null   float64
 10  BsmtQualNum          1459 non-null   int64  
 11  1stFlrSF             1459 non-null   int64  
 12  ExterScore           1459 non-null   int64  
 13  OverallScore         1459 non-null   int64  
 14  FullBath     

Successfully loaded HousePriceRegressionModel version 19
<class 'mlflow.pyfunc.PyFuncModel'>
Training columns: ['OverallQual', 'TotalSF', 'GrLivArea', 'GarageCapacityValue', 'ExterQualNum', 'GarageCars', 'GarageArea', 'TotalBath', 'TotalUsableBsmtSF', 'TotalBsmtSF', 'BsmtQualNum', '1stFlrSF', 'ExterScore', 'OverallScore', 'FullBath', 'BsmtQual_num', 'TotRmsAbvGrd', 'YearBuilt', 'KitchenQual_num', 'YearRemodAdd', 'Id']
Test columns:     ['OverallQual', 'TotalSF', 'GrLivArea', 'GarageCapacityValue', 'ExterQualNum', 'GarageCars', 'GarageArea', 'TotalBath', 'TotalUsableBsmtSF', 'TotalBsmtSF', 'BsmtQualNum', '1stFlrSF', 'ExterScore', 'OverallScore', 'FullBath', 'BsmtQual_num', 'TotRmsAbvGrd', 'YearBuilt', 'KitchenQual_num', 'YearRemodAdd']
Raw prediction range: 51020.59453133388 - 565221.2430991645
Final price range: 51020.59453133388 - 565221.2430991645
Submission file created successfully!


In [33]:

# pd.set_option('display.max_rows', None)

# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 1000)

# print(submission)

# # print("Sample of cleaned data:")
# # print(test_cleaned.head())
# # print("Columns:", test_cleaned.columns)

# # print("Sample of engineered data:")
# # print(test_engineered.head())
# # print("Columns:", test_engineered.columns)

# # print("Final features for prediction:")
# # print(test_features.head())
# # print("Feature statistics:")
# # for col in test_features.columns:
# #     print(f"{col}: min={test_features[col].min()}, max={test_features[col].max()}, mean={test_features[col].mean()}")
